# DRAGONWIND Scenario Comparison - Part 2: Running Simulations

This notebook continues from the scenario_comparison_part1 notebook, where we set up different scenarios.
Now we'll run the simulations for each scenario and analyze the results.

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Add the project root to the Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import DRAGONWIND modules
from src.config.loader import load_config
from src.core.simulation_engine import SimulationEngine
from src.scenarios.scenario_manager import Scenario, ScenarioManager
from src.analytics.plotter import create_figure
from src.utils.progress import ProgressTracker

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Define simulation timeframe
start_year = 2025
end_year = 2050

# Create a scenario manager and load the scenarios we created in part 1
scenario_manager = ScenarioManager()
scenarios = scenario_manager.list_scenarios()
print(f"Found scenarios: {scenarios}")

# If no scenarios were found, recreate them
if not scenarios:
    print("No scenarios found. Please run scenario_comparison_part1 notebook first.")
    # Code to recreate scenarios would go here
    # For brevity, we'll skip this

## Run Simulations for Each Scenario

Now, let's run simulations for each scenario including a baseline scenario.

In [ ]:
# Function to run a simulation for a given scenario
def run_scenario_simulation(scenario_name=None, start_year=2025, end_year=2050):
    """Run a simulation for the specified scenario."""
    # Load base configuration
    config = load_config()
    
    # Apply scenario overrides if specified
    if scenario_name is not None and scenario_name != "baseline":
        scenario = scenario_manager.load_scenario(scenario_name)
        if scenario:
            print(f"Applying scenario: {scenario_name} - {scenario.description}")
            config = scenario.apply_to_config(config)
        else:
            print(f"Warning: Scenario '{scenario_name}' not found. Using baseline configuration.")
    
    # Create and configure simulation engine
    engine = SimulationEngine(start_year=start_year, end_year=end_year)
    engine.configure(config)
    
    # Run the simulation
    print(f"Running simulation from {start_year} to {end_year}...")
    with ProgressTracker(total=end_year-start_year+1, description=f"Simulating {scenario_name or 'baseline'}") as progress:
        engine.run(progress=progress)
    
    print("Simulation completed!")
    
    # Get and return the results
    return engine.get_all_results()

In [ ]:
# Define the scenarios we want to run
scenario_names = ["baseline", "accelerated_growth", "grid_bottleneck", "manufacturing_constraint", "storage_integration"]

# Run simulations and store results
scenario_results = {}

for scenario_name in scenario_names:
    print(f"\n{'-'*80}\nRunning {scenario_name} scenario\n{'-'*80}")
    scenario_results[scenario_name] = run_scenario_simulation(
        scenario_name if scenario_name != "baseline" else None,
        start_year,
        end_year
    )

## Compare Renewable Capacity Expansion Across Scenarios

Let's compare how renewable energy capacity expands under different scenarios.

In [ ]:
# Extract and plot total renewable capacity for each scenario
plt.figure(figsize=(14, 8))

for scenario_name, results in scenario_results.items():
    if 'renewable' in results and results['renewable'] is not None:
        renewable_df = results['renewable'].reset_index()
        plt.plot(
            renewable_df['year'],
            renewable_df['total_capacity'], 
            linewidth=2.5 if scenario_name == "baseline" else 2.0,
            linestyle='-' if scenario_name == "baseline" else '--',
            marker='o' if scenario_name == "baseline" else None,
            markersize=5,
            label=f"{scenario_name.replace('_', ' ').title()}"
        )

plt.title('Total Renewable Energy Capacity by Scenario (2025-2050)', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Total Capacity (GW)', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

# Print the final year capacity for each scenario
print("Final Year (2050) Total Renewable Capacity by Scenario:")
for scenario_name, results in scenario_results.items():
    if 'renewable' in results and results['renewable'] is not None:
        final_capacity = results['renewable'].iloc[-1]['total_capacity']
        print(f"{scenario_name.ljust(25)}: {final_capacity:,.2f} GW")

## Compare Grid Integration Challenges

Now, let's examine how grid integration issues vary across scenarios.

In [ ]:
# Extract and plot curtailment rates for each scenario
plt.figure(figsize=(14, 8))

for scenario_name, results in scenario_results.items():
    if 'grid' in results and results['grid'] is not None:
        grid_df = results['grid'].reset_index()
        plt.plot(
            grid_df['year'],
            grid_df['curtailment_rate'] * 100,  # Convert to percentage 
            linewidth=2.5 if scenario_name == "baseline" else 2.0,
            linestyle='-' if scenario_name == "baseline" else '--',
            marker='o' if scenario_name == "baseline" else None,
            markersize=5,
            label=f"{scenario_name.replace('_', ' ').title()}"
        )

plt.title('Renewable Energy Curtailment Rate by Scenario (2025-2050)', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Curtailment Rate (%)', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

# Print the max curtailment rate for each scenario
print("Maximum Curtailment Rate by Scenario:")
for scenario_name, results in scenario_results.items():
    if 'grid' in results and results['grid'] is not None:
        max_curtailment = results['grid']['curtailment_rate'].max() * 100
        max_year = results['grid']['curtailment_rate'].idxmax()
        print(f"{scenario_name.ljust(25)}: {max_curtailment:.2f}% in year {max_year}")